In [1]:
import requests
import pandas as pd
import time 
from datetime import datetime

#Load Credentials
import json
# Load the configuration from config.json
with open('config.json') as config_file:
    config_data = json.load(config_file)
from sleeper_functions import *

In [2]:
matchup_df, updated_standings, HT_Owner, HT_Score, HP_Owner, HP_Player, HP_Score = run_sleeper_weekly(week = 2)

Loaded the League Data from Sleeper
Week 1 has been processed
Week 2 has been processed
Completed processing scores and updating standings
Saved week 2 matchup data


In [3]:
updated_standings

,owner,team_name,wins,losses,points_scored,lowest_scoring_team,median_weekly_score
4,Kyle,GOAT Dynasty,2,0,293,0,172.88
11,Andrew,King Drew,2,0,269,0,121.08
10,Dan,Prototypical NFL Corner,2,0,264,0,154.24
3,Henry,haamsuplo21,2,0,262,0,154.32
7,David,Korean Jesus,2,0,219,0,108.70
9,Kai,Not Sketch,1,1,237,0,116.26
1,Richard,RAVENS KICKERS & D & O,1,1,180,1,111.26
2,Johnny,ChubbHub,0,2,227,0,108.12
5,Scott,sc00t3r,0,2,222,0,105.16
8,Tom,"I Choose You, Pukachu",0,2,215,0,123.76


## II. Feed Data into GPT

In [18]:
import openai
openai.api_key = config_data['openai']['api_key']
#Feed in formatted data of the weekly matchups and standings and ask GPT to draft a weekly summary email to the league
from tabulate import tabulate
matchup_tabulate = tabulate(matchup_df, headers='keys', tablefmt='plain', showindex=False)
standings_tabulate = tabulate(updated_standings, headers='keys', tablefmt='plain', showindex=False)
print(matchup_tabulate)

owner      matchup_id    total_team_points  QB                 QB_PTS  RB1                    RB1_PTS  RB2                RB2_PTS  WR1                  WR1_PTS  WR2                WR2_PTS  TE                   TE_PTS  SF                    SF_PTS    FLEX    FLEX_PTS  K                  K_PTS  DEF      DEF_PTS
JD                  1               109.54  Tua Tagovailoa      13.26  Bijan Robinson            19.2  Derrick Henry         17    Mike Williams           12.6  Michael Pittman        9.6  Evan Engram             8.7  Tyler Allgeier           4.8    4892       18.38  Harrison Butker        5  NYJ            1
Andrew              1               121.08  Daniel Jones        31.74  Rhamondre Stevenson       13.5  Aaron Jones            0    Tyreek Hill             12.5  A.J. Brown             4.9  Cole Kmet               5.8  Tyler Lockett           21.9    9229       17.74  Cameron Dicker         8  SF             5
Johnny              2               108.12  Jared Goff          23.

In [25]:
input_prompt = f"Week:{week}, \nMatchup Table: \n\n{matchup_tabulate}\n\nStandings Table: \n\n{standings_tabulate}\n\nSummary:"

response2 = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "You are an AI Fantasy Football commissioner tasked with writing a weekly summary to your league mates recapping the latest week of our Dynasty league which is in it's first year.\n\nI will provide you a table of the weekly matchups, which includes the owners, their matchup_ids (owners with the same matchup IDs are opponents for the week), their players and what they scored, and a standings table with everyone's records. \nUsing this information, I would like for you to write an email recapping the league in the style of Bill Simmons. Comment on individual match-ups, teams and players that scored well, and poke fun at the worst performing teams. Make the tone funny, light-hearted and slightly sarcastic. "
    },
    {
      "role": "user",
      "content": input_prompt
    }
  ],
  temperature=1,
  max_tokens=2000,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response2['choices'][0]['message']['content'])

Hey there, 

Week 2 in our one-year-old Dynasty league is officially in the books, and boy, was it a doozy! We had some high-scoring matchups, some underperforming teams, and some, well, unfortunate sounding team names. Let's dive right into the action!

In the highlight matchup of the week, Andrew's "King Drew" continued his dominance by taking down JD's "Big Cheese." Andrew rode the arm of Daniel Jones, who somehow managed to put up 31.74 points (I didn't know he had that in him either) and Tyreek Hill's steady performance to secure a 121.08-109.54 victory. JD, on the other hand, couldn't quite keep up despite strong showings from Bijan Robinson and Derrick Henry. It seems the "Big Cheese" may have melted a bit this week.

Johnny's "ChubbHub" had a tough go of it as he fell to David's "Korean Jesus" in a battle that went down to the wire. Despite strong performances from Jared Goff and Saquon Barkley, Johnny couldn't overcome a receiving corps led by Justin Jefferson and a solid outi

## III. Draft Summary Email via Gmail

In [26]:
owners_dict = {
    'Peter': 'peteryushunli@gmail.com',
    'Richard': 'Richardliang2015@gmail.com',
    'Johnny': 'choi.john91@gmail.com',
    'Henry': 'henryzhang21@gmail.com',
    'Kyle': 'komearadoc@gmail.com',
    'Scott': 'lin.scott.bhs@gmail.com',
    'JD': 'johndong2015@u.northwestern.edu',
    'David': 'dchoi26273@gmail.com',
    'Tom': 'tzhangbhs@gmail.com',
    'Kai': 'Kaishen2@gmail.com',
    'Dan': 'jiangd93@gmail.com',
    'Andrew': 'Andydozntcare@gmail.com'
}

In [27]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Load the user name and password from yaml file
user, password = config_data['gmail']['GMAIL_USER'], config_data['gmail']['GMAIL_PW']

matchup = matchup_df.to_html(index=False)
standings = updated_standings.to_html(index=False)

# Initialize the server
s = smtplib.SMTP('smtp.gmail.com', 587)
s.starttls()
s.login(user, password)

# Define message
subject = f"Fagudas Dynasty FF Week {week} Recap"
summary = response2['choices'][0]['message']['content']
summary_html = summary.replace('\n', '<br>')
body = f"""\
<html>
  <head></head>
  <body>
    <h1>{subject}</h1>
    <br>{summary_html}</br>
    <br>
    <hr>
    <p>Highest Scoring Team: {HT_Owner}: {HT_Score} points</p>
    <p>Highest Scoring Player: {HP_Owner} - {HP_Player}: {HP_Score} Points</p>
    <br>
    {standings}
  </body>
</html>
"""

recipients = list(owners_dict.values())
sender = "peteryushunli@gmail.com"

msg = MIMEMultipart("alternative")
msg["From"] = sender
msg["To"] = ", ".join(recipients)
msg["Subject"] = subject

# Attach the HTML body
msg.attach(MIMEText(body, 'html'))

# Send email
s.sendmail(sender, recipients, msg.as_string())

# Terminate the session
s.quit()

(221,
 b'2.0.0 closing connection d3-20020a0caa03000000b00655e3e86ad7sm4834421qvb.6 - gsmtp')